### Features

Your goal is to predict the operating condition of a waterpoint for each record in the dataset. You are provided the following set of information about the waterpoints:

* `amount_tsh` :                        Total static head (amount water available to waterpoint)
* `date_recorded` :                     The date the row was entered
* `funder` :                            Who funded the well
* `gps_height` :                        Altitude of the well
* `installer` :                         Organization that installed the well
* `longitude` :                         GPS coordinate
* `latitude` :                          GPS coordinate
* `wpt_name` :                          Name of the waterpoint if there is one
* `num_private` :
* `basin` :                             Geographic water basin
* `subvillage` :                        Geographic location
* `region` :                            Geographic location
* `region_code` :                       Geographic location (coded)
* `district_code` :                     Geographic location (coded)
* `lga` :                               Geographic location
* `ward` :                              Geographic location
* `population` :                        Population around the well
* `public_meeting` :                    True/False
* `recorded_by` :                       Group entering this row of data
* `scheme_management` :                 Who operates the waterpoint
* `scheme_name` :                       Who operates the waterpoint
* `permit` :                            If the waterpoint is permitted
* `construction_year` :                 Year the waterpoint was constructed
* `extraction_type` :                   The kind of extraction the waterpoint uses
* `extraction_type_group` :             The kind of extraction the waterpoint uses
* `extraction_type_class` :             The kind of extraction the waterpoint uses
* `management` :                        How the waterpoint is managed
* `management_group` :                  How the waterpoint is managed
* `payment` :                           What the water costs
* `payment_type` :                      What the water costs
* `water_quality` :                     The quality of the water
* `quality_group` :                     The quality of the water
* `quantity` :                          The quantity of water
* `quantity_group` :                    The quantity of water
* `source` :                            The source of the water
* `source_type` :                       The source of the water
* `source_class` :                      The source of the water
* `waterpoint_type` :                   The kind of waterpoint
* `waterpoint_type_group` :             The kind of waterpoint

### Labels

There are three possible values:

* `functional` :                        The waterpoint is operational and there are no repairs needed
* `functional needs repair` :           The waterpoint is operational, but needs repairs
* `non functional` :                    The waterpoint is not operational


# Imports

In [170]:
# Standard DS modules
import featuretools as ft
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 200)

# Visualization modules
import altair.vegalite.v2 as alt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import export_graphviz
from graphviz import Source
from IPython.core.display import Image, display

# Model-related modules
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier    # Baseline
from xgboost import XGBClassifier


# Utilities
import stackprinter
stackprinter.set_excepthook(style='darkbg')

# Read CSVs & Verify Read Integrity

In [66]:
features_df = pd.read_csv('data/train_features.csv', infer_datetime_format=True)
labels_df = pd.read_csv('data/train_labels.csv')

In [67]:
features_df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [68]:
labels_df.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


# Data Exploration

### Numeric/Non-numeric descriptions

In [69]:
features_df.describe(include='number')

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [70]:
# Many of these features have low cardinality and are 
# a mixture of nominal and ordinal data
features_df.describe(exclude='number')

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,59400,55765,55745,59400,59400,59029,59400,59400,59400,56066,59400,55523,31234,56344,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400
unique,356,1897,2145,37400,9,19287,21,125,2092,2,1,12,2696,2,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6
top,2011-03-15,Government Of Tanzania,DWE,none,Lake Victoria,Madukani,Iringa,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,K,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
freq,572,9084,17402,3563,10248,508,5294,2503,307,51011,59400,36793,682,38852,26780,26780,26780,40507,52490,25348,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625


In [71]:
labels_df['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [72]:
features_df.corr()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
id,1.000000,-0.005321,-0.004692,-0.001348,0.001718,-0.002629,-0.003028,-0.003044,-0.002813,-0.002082
amount_tsh,-0.005321,1.000000,0.076650,0.022134,-0.052670,0.002944,-0.026813,-0.023599,0.016288,0.067915
gps_height,-0.004692,0.076650,1.000000,0.149155,-0.035751,0.007237,-0.183521,-0.171233,0.135003,0.658727
longitude,-0.001348,0.022134,0.149155,1.000000,-0.425802,0.023873,0.034197,0.151398,0.086590,0.396732
latitude,0.001718,-0.052670,-0.035751,-0.425802,1.000000,0.006837,-0.221018,-0.201020,-0.022152,-0.245278
num_private,-0.002629,0.002944,0.007237,0.023873,0.006837,1.000000,-0.020377,-0.004478,0.003818,0.026056
region_code,-0.003028,-0.026813,-0.183521,0.034197,-0.221018,-0.020377,1.000000,0.678602,0.094088,0.031724
district_code,-0.003044,-0.023599,-0.171233,0.151398,-0.201020,-0.004478,0.678602,1.000000,0.061831,0.048315
population,-0.002813,0.016288,0.135003,0.086590,-0.022152,0.003818,0.094088,0.061831,1.000000,0.260910
construction_year,-0.002082,0.067915,0.658727,0.396732,-0.245278,0.026056,0.031724,0.048315,0.260910,1.000000


# Instantiate Basline Estimator & Evaluate Accuracy

For this baseline I'm going to use a Decision Tree Classifier with all default parameters, though `random_state` is set for reproducibility. I didn't clean the data or engineer any features, so only features that are currently of numeric type will  be used to fit and score the estimator. The requirement for baseline accuracy is 60%, and for context the performant models score around 80% - 83%.

In [73]:
X_train, X_test, y_train, y_test = train_test_split(features_df, labels_df['status_group'], test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((47520, 40), (11880, 40), (47520,), (11880,))

In [74]:
clf = DecisionTreeClassifier(random_state=42)

clf.fit(X_train.select_dtypes('number'), y_train)
clf.score(X_test.select_dtypes('number'), y_test)

0.6531986531986532

In [75]:
# TODO: Visualize Decision Tree

# Make Predictions and Submit to Kaggle

In [76]:
test_features = pd.read_csv('data/test_features.csv')
y_pred = clf.predict(test_features.select_dtypes('number'))

sample_submission = pd.read_csv('data/sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('joseph_wagner_entry_1.csv', index=False)

# Encode Categorical Features

In [78]:
features_df.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq
recorded_by,59400,1,GeoData Consultants Ltd,59400
public_meeting,56066,2,True,51011
permit,56344,2,True,38852
source_class,59400,3,groundwater,45794
management_group,59400,5,user-group,52490
quantity_group,59400,5,enough,33186
quantity,59400,5,enough,33186
waterpoint_type_group,59400,6,communal standpipe,34625
quality_group,59400,6,good,50818
payment_type,59400,7,never pay,25348


In [79]:
features_df.select_dtypes(exclude='number').head()

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,2011-03-14,Roman,Roman,none,Lake Nyasa,Mnyusi B,Iringa,Ludewa,Mundindi,True,GeoData Consultants Ltd,VWC,Roman,False,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,2013-03-06,Grumeti,GRUMETI,Zahanati,Lake Victoria,Nyamara,Mara,Serengeti,Natta,NaN,GeoData Consultants Ltd,Other,NaN,True,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,2013-02-25,Lottery Club,World vision,Kwa Mahundi,Pangani,Majengo,Manyara,Simanjiro,Ngorika,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,2013-01-28,Unicef,UNICEF,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mahakamani,Mtwara,Nanyumbu,Nanyumbu,True,GeoData Consultants Ltd,VWC,NaN,True,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,2011-07-13,Action In A,Artisan,Shuleni,Lake Victoria,Kyanyamisa,Kagera,Karagwe,Nyakasimbi,True,GeoData Consultants Ltd,NaN,NaN,True,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [178]:
def wrangle(X):
    df = X.copy()
    le = LabelEncoder()

    # Impute missing observations
    df.fillna(method='bfill', inplace=True)
    
    # Drop recorded_by: 1 unique value, predictive power
    # Drop wpt_name: majority unique values, most frequent value is 'none'
    # Drop extraction_type_group, waterpoint_type_group, etc: extraction_type has the same data, with more cardinality
    # Drop subvillage, because its very high cardinality, and I think numeric lat/long data will be 
    # interpreted by the model more effectively than 20,000 unique ints without context
    # Drop region: region_codes exists
    df = df.drop(columns=['id', 'recorded_by','wpt_name', 'extraction_type_group',
                          'extraction_type_class','waterpoint_type_group','management',
                          'payment','quantity_group','region','source_type',
                          'subvillage','scheme_name','funder','date_recorded'])


    nominals = ['installer', 'basin', 'lga', 'ward', 'public_meeting', 
                'scheme_management', 'permit', 'extraction_type', 'management_group', 
                'payment_type', 'water_quality', 'source', 'waterpoint_type', 
                'quality_group', 'quantity', 'source_class',]
    
    for feature in nominals:
        df[feature] = le.fit_transform(df[feature])
    
    return df

In [179]:
X_train_wrangled = wrangle(X_train)
X_test_wrangled = wrangle(X_test)
X_train_wrangled.head()

,amount_tsh,gps_height,installer,longitude,latitude,num_private,basin,region_code,district_code,lga,ward,population,public_meeting,scheme_management,permit,construction_year,extraction_type,management_group,payment_type,water_quality,quality_group,quantity,source,source_class,waterpoint_type
3607,50.0,2092,335,35.426020,-4.227446,0,0,21,1,2,19,160,1,10,1,1998,3,4,5,6,2,2,8,0,1
50870,0.0,0,505,35.510074,-5.724555,0,0,1,6,4,842,0,1,7,1,0,4,4,2,6,2,1,7,0,4
20413,0.0,0,720,32.499866,-9.081222,0,2,12,6,62,1572,0,1,7,0,0,9,4,2,6,2,1,7,0,6
52806,0.0,0,774,34.060484,-8.830208,0,6,12,7,59,176,0,1,7,1,0,3,4,1,6,2,2,6,1,1
50091,300.0,1023,774,37.032690,-6.040787,0,8,5,1,36,143,120,1,7,1,1997,9,4,3,4,4,1,7,0,6


# Baseline Model Score 2: After Data Cleaning

In [180]:
clf_enc = DecisionTreeClassifier(random_state=42)

clf_enc.fit(X_train_wrangled, y_train)
clf_enc.score(X_test_wrangled, y_test)

0.7183501683501684

In [135]:
test_features_wrangled = wrangle(test_features)
y_pred = clf_enc.predict(test_features_wrangled)

sample_submission = pd.read_csv('data/sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('joseph_wagner_entry_2.csv', index=False)

# Hyperparameter Search for XGBoost Classifier

In [182]:
X_train_wrangled.shape, y_train.shape

((47520, 25), (47520,))

In [183]:
search_space = {'max_depth': [2,3,4,5,6,7,8,10],
                'learning_rate': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
                'n_estimators': [80,100,110,125,135,150,175,200,250],
                'gamma': [0,0.2,0.5,0.8,1,1.3,1.7,2.0,2.5,3,4,5],
                'min_child_weight': [1,2,3,4,5],
}

opt = RandomizedSearchCV(
    XGBClassifier(objective='multi:softmax', num_class=3, random_state=42, n_jobs=-1, verbose=3),
    param_distributions=search_space,
    n_iter=30,
    cv=3,
    verbose=3,
    random_state=42,
    n_jobs=-1
)

opt.fit(X_train_wrangled, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 14.1min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=-1, nthread=None, num_class=3,
                                           objective='multi:softmax',
                                           random_state=42, reg_alpha=0,
                                           reg_la...
                   iid='warn', n_iter=30, n_jobs=-1,
                   param_distributions={'gamma': [0, 0.2, 0.5, 0.8, 1, 1.3, 1.7,
                                                  2.0, 2.5, 3, 4, 5],
                 

In [184]:
best = opt.best_estimator_

In [185]:
pd.Series(best.feature_importances_, X_train_wrangled.columns)

amount_tsh           0.020549
gps_height           0.101543
installer            0.063282
longitude            0.170371
latitude             0.178453
num_private          0.003095
basin                0.009845
region_code          0.014230
district_code        0.020463
lga                  0.039250
ward                 0.089205
population           0.065431
public_meeting       0.004213
scheme_management    0.017712
permit               0.007437
construction_year    0.049998
extraction_type      0.031469
management_group     0.009630
payment_type         0.019518
water_quality        0.010146
quality_group        0.004600
quantity             0.024290
source               0.020635
source_class         0.005417
waterpoint_type      0.019217
dtype: float32

# XGBoost Score 1

In [186]:
y_pred = best.predict(test_features_wrangled)

sample_submission = pd.read_csv('data/sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('joseph_wagner_entry_3.csv', index=False)

* Try to utilize Lat/Long/Height features
* Visualize/Interpret Model